# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [12]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [13]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'platform / Nebula',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/'},
  {'type': 'blog index', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blo

In [15]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [16]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [17]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [18]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
PaddlePaddle/PaddleOCR-VL
Updated
1 day ago
•
3.82k
•
638
nanonets/Nanonets-OCR2-3B
Updated
4 days ago
•
12.8k
•
323
Qwen/Qwen3-VL-8B-Instruct
Updated
4 days ago
•
74.5k
•
198
Phr00t/Qwen-Image-Edit-Rapid-AIO
Updated
1 day ago
•
356
inclusionAI/Ring-1T
Updated
6 days ago
•
435
•
175
Browse 1M+ models
Spaces
Running
325
325
veo3.1-fast
🐨
Generate videos from text or images
Running
15.2k
15.2k
DeepSite v3
🐳
Gene

In [28]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nPaddlePaddle/PaddleOCR-VL\nUpdated\n1 day ago\n•\n3.82k\n•\n638\nnanonets/Nanonets-OCR2-3B\nUpdated\n4 days ago\n•\n12.8k\n•\n323\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n4 days ago\n•\n74.5k\n•\n198\nPhr00t/Qwen-Image-Edit-Rapid-AIO\nUpdated\n1 day ago\n•\n356\nincl

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


# Hugging Face – The AI Community Building the Future

---

## About Us

Hugging Face is the premier collaboration platform for the global machine learning (ML) community. We empower engineers, scientists, and end users to create, share, explore, and innovate in the field of AI through open-source models, datasets, and applications.

Our platform serves as a central hub where over a million machine learning models, hundreds of thousands of datasets, and hundreds of thousands of applications are hosted and actively used, making it the home of machine learning collaboration worldwide.

---

## Our Platform

- **Models:** Access and contribute to over 1 million public ML models across text, image, video, audio, and even 3D modalities. From optical character recognition to advanced image editing and instruction-based models, our repository is a treasure trove of AI innovation.
  
- **Datasets:** Browse more than 250,000 datasets curated for various AI research needs, enabling easy experimentation and accelerated development.
  
- **Spaces:** Build and deploy AI applications in real-time. With over 400,000 applications hosted, Hugging Face Spaces enable developers to showcase interactive ML demos, driving adoption and community feedback.
  
- **Community:** Join a growing, fast-moving community of AI practitioners collaborating on open-source projects. Share your work, build your portfolio, and accelerate AI development.

- **Enterprise Solutions:** We also provide advanced paid Compute and Enterprise services to help teams and organizations accelerate AI innovation efficiently.

- **HuggingChat Omni:** Try our cutting-edge multi-modal chat AI ready for immediate inference.

---

## Our Culture

At Hugging Face, we believe in **openness, collaboration, and ethical AI development**. Our community-first approach fosters an environment where ideas are shared freely, and innovation flourishes without constraints.

We emphasize:
- **Inclusivity:** Welcoming ML practitioners of all levels to contribute and learn.
- **Transparency:** Open sourcing our tools and resources to build trust in AI.
- **Innovation:** Rapid iteration and sharing to push the boundaries of artificial intelligence.
- **Support:** Providing comprehensive docs, community channels, and enterprise-grade support to empower our users.

As the AI community builds the future, Hugging Face stands out as the platform that brings the world of machine learning together.

---

## Customers & Partners

Our user base spans independent ML engineers, academic researchers, startups, and large enterprises seeking the most advanced AI tools and collaboration platforms.

- Startups and AI builders use Reachy Mini, our open robot, to prototype AI-driven applications.
- Leading companies leverage our enterprise platform for scalable AI deployment.
- Researchers and data scientists contribute to and benefit from the rich datasets and models hosted on our Hub.

---

## Careers at Hugging Face

Join a dynamic and fast-growing team shaping the AI future. We seek talented engineers, AI researchers, product managers, and community specialists passionate about open source, ethics, and collaboration.

**Why work with us?**

- Impact the future of AI through open and ethical innovation.
- Work closely with a global ML community.
- Be part of an inclusive and supportive culture.
- Opportunities for remote and flexible working environments.

Visit our website for current openings and to become part of the AI revolution.

---

## Connect With Us

- **Website:** [huggingface.co](https://huggingface.co)
- **Explore models, datasets, and apps** at our Hub.
- **Join our community** and start collaborating today.
- **Try HuggingChat Omni** and experience conversational AI.
- **Follow us on social media** for the latest news and updates.

**Hugging Face**  
Building the future of AI — together.

---

*Brand Colors:*  
Yellow (#FFD21E, #FF9D00), Gray (#6B7280)  
Available Logos: SVG, PNG, AI formats

---

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Hugging Face: The AI Community Building the Future (and Hugging You with Code!)

Welcome to **Hugging Face**, where Machine Learning goes from "Hmm, what if?" to "Wow, what now?!" faster than you can say "Open source!" We’re not just a company—we’re a vibrant, buzzing community of ML enthusiasts, engineers, scientists, artists, and robots (okay, Reachy Mini for now) building the future—together.

---

## What’s Hugging Face All About?

Imagine a place where over **1 million models**, **250,000 datasets**, and **400,000+ AI applications** hang out, collaborate, and upgrade the AI universe. That’s us! Whether you want to generate videos, decode handwritten texts, chat with an AI, or create the next AI-powered gadget, Hugging Face is your playground.

Our motto: **"Collaborate, Share, Innovate, Repeat."** Because AI is better when it’s not solo.

---

## Hugging Face Highlights:

- **Models Galore:** From PaddleOCR that can read your handwritten shopping list to Qwen3-VL that’s basically an AI Picasso – explore, use, or build upon them all.
- **Spaces to Shine:** Host your AI experiments – videos, images, audios, even 3D applications – in public or private Spaces that hum with community energy.
- **Datasets Feast:** Dive into datasets like ChatGPT prompts or massive Webscale Reinforcement Learning data to fuel your AI hunger.
- **HuggingChat Omni:** Try chatting with AI that has the wit of a stand-up comedian and the knowledge of a scholar.
- **Reachy Mini:** Our open-source robot buddy, perfect for AI builders who want a tactile team member.

---

## Culture: Hugging with Code (and Heart)

- **Community-first:** We’re not just code; we're care. AI for all, by all.
- **Open & Ethical:** Transparency is our jam. Open-source everything, ethical AI forever.
- **Collaborative Spirit:** Whether you’re a rookie or a guru, jump in, share your work, and grow your portfolio.
- **Innovation Playground:** Experiment freely. Crash, reboot, innovate. Repeat till you’re proud.
- **Fun & Friendly:** Expect more emojis than a texting teenager and a lot of 🐨 pandas and 🤖 robots (virtual, for now).

---

## For Our People—Careers That Hug You Back!

Love ML? Want to build the tools that the world's smartest minds use every day? Hop aboard! Join a team where:

- Your code is not confined to cubicles but shared with millions.
- You enable AI breakthroughs that might shape the next decade.
- You get to hang out with AI bots like Reachy and chatbots with personality.
- Growth, learning, and impact come standard.
  
Check out our jobs, pitch your dream role, or bring your unique AI flavor to our global, inclusive machine learning party.

---

## Our Customers & Collaborators

From solo researchers and educators to enterprise teams turbocharging their AI capabilities, Hugging Face is the go-to hub:

- ML engineers craving a rich model and dataset playground.
- AI startups accelerating innovations with open-source magic.
- Enterprises tapping our platform for scalable AI solutions.
- Creators pushing the boundary of AI applications.
  
---

## Why Hugging Face?

Because AI’s future should be **open, ethical, creative, and community-powered**—and maybe just a little cuddly. We give you the fastest route to AI innovation, robust tools, and the friendliest global ML crew cheering you on.

Ready to get hugged by ML? Visit [huggingface.co](https://huggingface.co) to join the biggest, most vibrant machine learning community on the planet.

---

## Hugging Face at a Glance

| Feature           | Details                              |
|-------------------|------------------------------------|
| Models            | 1M+ Open-source models              |
| Datasets          | 250K+ public datasets               |
| Applications      | 400K+ AI applications under "Spaces"|
| Community         | Millions of ML collaborators globally|
| Special Projects  | HuggingChat Omni, Reachy Mini robot |
| Company Colors    | #FFD21E (Sunny Hug Yellow), #FF9D00 (Warm Hug Orange) |
| Logo & Branding   | Hug your code with our smiley face!|

---

**Hugging Face** – Where AI meets heart, one model at a time. Come for the tech, stay for the hugs. 🤗

---

*Disclaimer: No pandas or robots were harmed in the making of this AI community. Only hugs given, models shared, and minds expanded.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>